# Residue Hyperdimensional Computing

In this notebook we implement Kymn et al. (2023), 
[*Computing with Residue Numbers in High-Dimensional Representation*](https://arxiv.org/abs/2311.04872).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import typing as t
from mypy_extensions import KwArg
import cmath
import math
from vsa.vocabulary import Vocabulary
from functools import reduce

## Residue Number System

A **Residue Number System** is an encoding scheme for integers $x \in \mathbb{Z}$
by $x$'s value modulo $[m_1, m_2, \ldots, m_K]$, where the $m_K$ are the 
moduli of the system.

For example, $x=20$, for moduli $[3, 5, 7]$, we have the encoding,
$$
[20 \bmod 3, 20 \bmod 5, 20 \bmod 7] = [2, 0, 6]
$$

If the moduli are pairwise co-prime, then for any $x$ s.t. 
$0 \leq x \leq M := \prod_k m_k$, the integer is uniquely encoded by the 
residue, by the Chinese Remainder Theorem.

In [2]:
def residue_encode(moduli: np.ndarray, x: int) -> np.ndarray:
    """
    Encode a number `x` by moduli `moduli`.
    """
    mod = np.vectorize(lambda n: x % n)
    return mod(moduli)


x = 20
moduli = np.array([3, 5, 7])
residue_encode(moduli=moduli, x=x)

array([2, 0, 6])

# Fractional Power Encoding

A **Fractional Power Encoding** (FPE) is a randomized mapping from an integer
$x$ to a high-d vector $\textbf{z}(x)$. Let $D$ be the dimension of $\textbf{z}(x)$,
where $D$ in the range of $10^2 \leq D \leq 10^4$.

To encode an integer, the first step is to draw a random base vector,
$\textbf{z}$, which is defined as follows,
$$
\textbf{z} = [e^{i\phi_1x}, e^{i\phi_2x}, \ldots, e^{i \phi_D x}]
$$
where each $e^{i\phi_j}$ is a phasor, and each $\phi_j$ is a random sample
from a probability distribution.

Then, we define a function $x \to \mathbb{C}^D$ via element-wise exponentiation
of the random vector.

In [40]:
def fpe(
    x: int,
    dim: int,
    phis: np.ndarray = None,
    dist: t.Callable[[KwArg], np.ndarray] = np.random.normal,
) -> t.Tuple[np.ndarray, np.ndarray]:
    """
    `fpe`
    Fractional power encoding

    Parameters:
    - x : int, the number to be encoded
    - dim : int, the dimension of the vector
    - samples : np.ndarray, the `phis` of the FPE
    - dist : t.Callable[[KwArg], np.ndarray], a probability distribution 
      function that should take a `size` kw argument to specify the dimension of 
      a vector of random values sampled from that distribution

    Returns:
        tuple of (encoded vector, phis)
    """
    I = cmath.sqrt(-1)
    
    def make_vectorized_fpe(x: int):
        return np.vectorize(lambda phi: I * phi * x)

    v_exp = make_vectorized_fpe(x)

    if phis is None:
        phis = dist(size=dim)
    vec = v_exp(phis)

    return vec, phis

vec, phis = fpe(10, dim=20)
print(f"Resulting vector, {vec}")
print(f"phis used, {phis}")

vec2, _ = fpe(20, dim=20, phis=phis)
np.dot(np.conjugate(vec).T, vec2).real / vec.size

Resulting vector, [0.-16.76771691j 0. +0.13098878j 0.+13.54445128j 0. -3.03573447j
 0. -0.56231124j 0.-10.05234726j 0. +2.27151824j 0. -6.32484545j
 0.-14.35876886j 0.+19.54884079j 0. +2.41268508j 0.+13.35801545j
 0. -7.59521186j 0.+28.41580326j 0. +2.04132713j 0. -3.01078148j
 0. -2.40629116j 0.+26.03882646j 0. -6.67369731j 0. -6.9200032j ]
phis used, [-1.67677169  0.01309888  1.35444513 -0.30357345 -0.05623112 -1.00523473
  0.22715182 -0.63248455 -1.43587689  1.95488408  0.24126851  1.33580155
 -0.75952119  2.84158033  0.20413271 -0.30107815 -0.24062912  2.60388265
 -0.66736973 -0.69200032]


np.float64(304.75720627799086)

# The Kernel

A **kernel** is a function $K(x_1, x_2)$ which measures the similarity between
two objects. We can induce a kernel based on the inner product of FPEs,
$$
K(x_1, x_2) = \frac{1}{D} \text{Re}(\textbf{z}(x_1)^T \overline{\textbf{z}(x_2)})
$$
where $\overline{\textbf{z}(x_2)}$ is the complex conjugate of $\textbf{z}(x_2)$.

In [4]:
def kernel(x_1: np.ndarray, x_2: np.ndarray) -> float:
    """
    `kernel`

    Arguments:
        - x_1 and x_2 : np.ndarray, FPEs with the same underlying distribution

    Returns:
        the similarity between the two
    """
    x_1T = x_1.T
    x_2conj = np.conjugate(x_2)
    x_1innerx_2 = np.inner(x_1T, x_2conj)
    return (x_1innerx_2.real) / x_1.size

x_1, phis = fpe(100, dim=100)
x_2, _ = fpe(100, dim=100, phis=phis)
kernel(x_1, x_2)

np.float64(11277.879593582977)

# Fractional Power Encoding, modulo $m$

For $\textbf{z}_m$, let the support of our probability distribution for $\phi_j$
be the $m$-th roots of unity. In other words, $\phi_j$ is a multiple of 
$2 \pi / m$. The congruent values are mapped to the same vector,
$$
\begin{align*}

\textbf{z}_m (x + m) &= [e^{i \phi_1 (x + m)}, e^{i \phi_2 (x + m)}, \ldots,
e^{i \phi_D (x + m)}] \\
&= [e^{i \phi_1 x} \cdot e^{i \phi_1 m}, e^{i \phi_2 x} \cdot e^{i \phi_2 m},
\ldots, e^{i \phi_D x} \cdot e^{i \phi_D m}] \\
&= [e^{i \phi_1 x}, e^{i \phi_2 x}, \ldots, e^{i \phi_D x}] \\
&= \textbf{z}_m (x)

\end{align*}
$$

because $e^{i \pi_j m} = e^{2 \pi \cdot i \cdot k}$ for some integer $k$,
and $e^{2\pi \cdot i \cdot k}$ for any integer $k$. This means that 
$\textbf{z}_m (x)$ is a *representation* of the additive group of integers 
modulo $m$.

The kernel induced for $\textbf{z}_m (\Delta x)$ is $1$ if 
$\Delta x = 0 \pmod m$, $\approx 0$ otherwise. This gets us quasi-orthogonality.


In [5]:
def roots_of_unity(m: int, dim: int) -> np.ndarray:
    """
    `roots_of_unity`

    Generate `m` roots of unity of size `dim`.
    """
    incr = 2 * math.pi / m
    points_of_unity = []
    curr_value = incr
    while curr_value < 2 * math.pi:
        points_of_unity.append(curr_value)
        curr_value += incr
    
    v = np.zeros(dim)
    choose = np.vectorize(lambda _: np.random.choice(points_of_unity))
    return choose(v)


def fpe_mod_m(
    x: int,
    m: int,
    dim: int,
    phis: np.ndarray | None = None,
) -> t.Tuple[np.ndarray, np.ndarray]:
    # generate phis from m roots of unity of the prob distribution
    if phis is None:
        phis = roots_of_unity(m, dim)
    
    vexp = np.vectorize(lambda n: cmath.exp(cmath.sqrt(-1) * x * n))
    return vexp(phis)


def fpe_mod_m_phi(m: int, dim: int) -> np.ndarray:
    phis = roots_of_unity(m, dim)
    vexp = np.vectorize(lambda n: cmath.exp(cmath.sqrt(-1) * n))
    return vexp(phis)


In [6]:
fpe_mod_m(20, 5, 10)

array([1.-1.95943488e-15j, 1.-2.93915232e-15j, 1.-3.91886976e-15j,
       1.-2.93915232e-15j, 1.-2.93915232e-15j, 1.-3.91886976e-15j,
       1.-3.91886976e-15j, 1.-9.79717439e-16j, 1.-9.79717439e-16j,
       1.-3.91886976e-15j])

# Residue Hyperdimensional Computing

Let $z_{m_1}, z_{m_2}, \ldots, z_{m_K}$ denote FPE vectors with moduli
$m_1, m_2, \ldots, m_K$. Let $\odot$ denote the Hadamard product. Then,
we can encode an integer $x$ by combining our modulo representations
via the Hadamard product,
$$
z (x) = \bigodot^K_{k=1} z_{m_k} (x)
$$

Below, we'll define a vector symbolic algebra for it

In [7]:
def rhc_ker(x: np.ndarray, y: np.ndarray) -> float:
    """
    `rhc_ker`

    Estimated kernel for Residue Hyperdimensional Computing (RHC).
    """
    assert x.size == y.size, "size equivalence in dot"
    return np.dot(x.T, y) / x.size



def rhc_mult(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """
    `rhc_mult`

    The second RHC binding operation, which implements multiplication
    over the algebra.
    """



## Additive Binding

We define additive binding by simple element-wise multiplication

In [8]:
rhc_bind = np.multiply
"""
`rhc_bind`

The first RHC binding operation is just element-wise multiplication; 
this implements addition over the algebra.
""";


## Multiplicative binding

The goal here is to define a binding operation $\star$, such that
$$
z (x_1 \cdot x_2) = z(x_1) \star z(x_2).
$$ 
If we had $x_2$ directly,
then we can just directly encode it by element-wise exponentiation of
$z(x_1)^{x_2}$.

We can do this without having to decode the right-hand side operand. Recall,
if $x$ is an integer, then each component of $z_{m_k}(x)$ is the $m_k$-th
root of unity. Moreover, $z_{m_k} (x) = e^{i \frac{2\pi}{m_k}r_j}$, for
$r_j \in \mathbb{Z}, r_j = \frac{m_k}{2\pi}\phi_j x \pmod{m_k}$

Therefore, the goal is to define an $f$, such that,
$$
f (e^{i \frac{2\pi}{m_k}r}, e^{i \frac{2\pi}{m_k}s}) = e^{i \frac{2\pi}{m_k}rs}
$$
But, if $r = \frac{m_k}{2\pi}\phi x_1$, and $s = \frac{m_k}{2\pi}\phi x_2$,
then $rs = \frac{m_k}{2\pi}\phi^2 x_1 x_2$, which is off from the desired
multiplicative factor. Therefore, we need to somehow cancel the extra fractor.

To do this, recall each phase $\phi$ is drawn from the $m$-th roots of unity,
and can be written $\frac{2\pi}{m_k}u$, where $u \in \mathbb{Z} \pmod{m_k}$.
When $m_k$ is prime, then any non-zero integer $u$ has a unique
modular inverse 
$v \in \mathbb{Z} \pmod m_k$, such that $u \times v = 1 \bmod{m_k}$.
E.g., the modular inverse of $\left[ 3 \pmod 5 \right]^{-1} = 2$. Therefore,
$$
f (e^{i\frac{2\pi}{m_k}u^2}, e^{i \frac{2\pi}{m_k}v}) = e^{i \frac{2\pi}{m_k}u}
$$
Therefore, we assume that for whenever multiplicative binding is used,
every moduli $m_k$ is prime. This allows us to define an ant-base vector
$y_{m_k}$ whose components are defined by the modular multiplicative
inverse of $z_{m_k}$. That is, if the $j$-th component of $z_{m_k}$ is
$e^{i \frac{2\pi}{m_k}u}$, then the $j$-th component of $y_{m_k}$ is 
$e^{i \frac{2\pi}{m_k}v}$.

In [23]:
def fpe_encode(self, n: int) -> np.ndarray:
    """
    `fpe_encode`

    Using some Python magic to shove encode into the object.
    """
    bases = [value for key, value in self.symbols.items() if "base" in key]
    dim = bases[0].size
    return reduce(
        lambda x, y: self.bind(x**n, y**n),
        bases,
        np.ones_like(dim)
    )





# we're going to be using moduli [3, 5, 7, 11]
dim = 600
symbols = {
    "base3": fpe_mod_m_phi(3, dim),
    "base5": fpe_mod_m_phi(5, dim),
    "base7": fpe_mod_m_phi(7, dim),
    "base11": fpe_mod_m_phi(11, dim),
}
rhc = Vocabulary(
    dim=dim,
    symbols=symbols,
    bind=rhc_bind,
    sim=rhc_ker,
    funcs={
        "encode": fpe_encode
    }
)

In [24]:
rhc.encode(rhc, n=3)

array([ 0.95230938-0.30513413j,  0.98923256+0.1463521j ,
       -0.61708939+0.78689306j,  0.7843688 -0.62029476j,
        0.93623487-0.35137482j,  0.99148918-0.13018909j,
       -0.33220389+0.9432076j ,  0.82316211-0.56780643j,
       -0.94978782+0.31289471j, -0.20658122-0.97842946j,
        0.98088465-0.1945901j ,  0.8667046 +0.49882175j,
        0.2778155 -0.96063445j,  0.99787003+0.06523349j,
        0.16650055-0.98604136j,  0.93623487-0.35137482j,
       -0.97582768+0.2185414j ,  0.6486721 -0.76106801j,
       -0.85423501-0.51988705j,  0.99946736-0.03263413j,
        0.85844879+0.51289928j,  0.83231866+0.55429744j,
        0.85844879-0.51289928j, -0.95476754+0.29735323j,
        0.83231866-0.55429744j,  0.32449636-0.94588694j,
       -0.0774422 -0.99699684j,  0.76370824+0.64556156j,
       -0.19058649-0.98167041j,  0.89007853-0.4558072j ,
        0.99520969-0.09776337j, -0.89376824-0.44852908j,
       -0.42282368-0.90621197j, -0.50939256+0.86053427j,
        0.67316338+0.73949379j,